In [ ]:
!pip install transformers datasets accelerate bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!pip install python-docx pypdf
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 22.2 MB/s eta 0:00:00


In [ ]:
import os
import re
import json
import gc
import pickle
import torch
from google.colab import files
from datasets import Dataset, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    logging as hf_logging,
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LC_Doc
from docx import Document as DocxDocument

# Silence HF warnings
hf_logging.set_verbosity_error()

# 1) Mitigate GPU fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [ ]:
# 2) Upload and parse instruction JSON
uploaded = files.upload()
instr_fname = next(iter(uploaded))
raw = uploaded[instr_fname].decode("utf-8")
# extract JSON array
start, end = raw.find("["), raw.rfind("]") + 1
js = raw[start:end]
js = re.sub(r",\s*\]$", "]", js)  # remove trailing comma
instr_pairs = json.loads(js)
instr_ds = Dataset.from_list([
    {"text": f"User: {ex['user_query']}\nAssistant: {ex['response']}\n"}
    for ex in instr_pairs
])

Saving Ecommerce and Retail Store Queries.txt to Ecommerce and Retail Store Queries.txt


In [ ]:
# 3) Upload and read Word document
uploaded = files.upload()
docx_fname = next(iter(uploaded))
doc = DocxDocument(docx_fname)
raw_text = "\n".join(p.text for p in doc.paragraphs)

Saving Ecom overview.docx to Ecom overview.docx


In [ ]:
def clean_healthcare_text(text: str) -> str:
    text = re.sub(r'\xa0(\d+)', r' \1', text)                     # Fix nbsp before numbers
    text = re.sub(r'(?<=\D)\$(\d)', r' \1', text)                 # Space before currency
    text = re.sub(r'\b(\d+)\s+([KM])\b', r'\1\2', text)           # 150 K → 150K
    text = re.sub(r'\$(\d+)\s+([KM])\b', r'\1\2', text)           # $10 K → $10K
    text = re.sub(r'(\d)(vs\.?)(\d)', r'\1 vs \3', text)          # 75(vs.150 → 75 vs 150
    text = re.sub(r'(\d)\s?–\s?(\d)', r'\1-\2', text)             # En-dash formatting
    text = re.sub(r'(\d+)([A-Za-z]{2,})', r'\1 \2', text)         # 50Kper → 50K per
    text = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', text)              # letters next to digits
    text = re.sub(r'\b(e\.)\s?(g\.)\s?,', r'e.g.,', text)         # normalize e.g.,
    text = re.sub(r'\b(i\.)\s?(e\.)\s?,', r'i.e.,', text)         # normalize i.e.,
    text = re.sub(r'\s+', ' ', text)                              # collapse whitespace
    text = re.sub(r'(?<=\D)\.(?=\S)', '. ', text).strip()         # ensure space after sentences
    text = re.sub(r'[^A-Za-z0-9\s]+', ' ', text)                  # remove all punctuation/symbols
    text = re.sub(r'\s+', ' ', text).strip()                      # collapse spaces again
    return text

In [ ]:
cleaned = clean_healthcare_text(raw_text)


In [ ]:
cleaned

'E Commerce Digital Retail Platforms Overview E commerce revolutionizes retail by digitizing transactions logistics and customer engagement It leverages AI blockchain and omnichannel strategies to deliver personalized shopping experiences Dominated by giants like Amazon global marketplaces Alibaba B 2B wholesale and Shopify SME storefronts the sector faces challenges like cart abandonment 70 average rate and supply chain volatility 1 Online Marketplaces B 2C Platforms General Marketplaces Amazon sells 12M products using AI recommendations to drive 35 of revenue Prime membership 200M users offers free shipping and streaming perks Niche Platforms Etsy focuses on handmade vintage goods with seller fees of 6 5 payment processing StockX authenticates sneakers via blockchain serialization Subscription Models Walmart 12 95 month includes fuel discounts and Scan Go in store tech B 2B Platforms Wholesale Trading Alibaba connects manufacturers with bulk buyers Trade Assurance escrow holds paymen

In [ ]:
# 5) Chunk into ~800-char slices
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
chunks = splitter.split_text(cleaned)
content_ds = Dataset.from_list([{"text": c} for c in chunks])

In [ ]:
# 1) Load tokenizer & base model in FP16
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/gpt-neo-1.3B",
    device_map="auto",
    torch_dtype=torch.float16,   # 16-bit precision
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [ ]:
# 2) Prepare model for LoRA:
#    - disable caching for checkpointing compatibility
#    - enable gradient checkpointing
#    - freeze all base-model parameters
model.config.use_cache = False
model.gradient_checkpointing_enable()
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# 3) Inject LoRA adapters (only these will train)
lora_cfg = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=16,               # LoRA rank
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_cfg)

In [ ]:
# Verify that only LoRA params are trainable
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,}")

Trainable params: 3,145,728 / 1,318,721,536


In [ ]:
def tokenize_and_label(ex):
    tok = tokenizer(
        ex["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )
    # For causal LM, labels = copy of input_ids
    tok["labels"] = tok["input_ids"].copy()
    return tok

tok_instr   = instr_ds.map(tokenize_and_label, batched=True, remove_columns=["text"])
tok_content = content_ds.map(tokenize_and_label, batched=True, remove_columns=["text"])

Map:   0%|          | 0/1170 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
train_dataset = concatenate_datasets([tok_instr, tok_content])

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
from transformers import DataCollatorForLanguageModeling
# 5) Use a data collator that handles causal LM labels
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


In [ ]:
# 5) Training arguments (FP16 + no eval split)
training_args = TrainingArguments(
    output_dir="./gpt_neo_fp16_lora",
    num_train_epochs=6,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    fp16=True,
    optim="adamw_torch",
    learning_rate=2e-4,
    weight_decay=0.01,
    max_grad_norm=1.0,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    eval_strategy="no",
    save_steps=200,
    save_total_limit=3,
    logging_steps=50,
    report_to="none",
    load_best_model_at_end=False,
)

In [ ]:
# 6) Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


In [ ]:
torch.cuda.empty_cache()
gc.collect()

238

In [ ]:
# 7) Train
trainer.train()
# 8) Save LoRA adapters & tokenizer
model.save_pretrained("./gpt_neo_fp16_lora")
tokenizer.save_pretrained("./gpt_neo_fp16_lora")
print("✅ FP16 + LoRA fine-tuning complete.")

{'loss': 3.9977, 'grad_norm': 10.92226791381836, 'learning_rate': 9.662921348314608e-05, 'epoch': 0.338409475465313}
{'loss': 0.2861, 'grad_norm': 0.0735745057463646, 'learning_rate': 0.00019998763234899838, 'epoch': 0.676818950930626}
{'loss': 0.1925, 'grad_norm': 0.07770732790231705, 'learning_rate': 0.00019775440433647662, 'epoch': 1.0135363790186125}
{'loss': 0.1637, 'grad_norm': 0.08832208067178726, 'learning_rate': 0.00019175515045539293, 'epoch': 1.3519458544839256}
{'loss': 0.1527, 'grad_norm': 0.08285439014434814, 'learning_rate': 0.00018222099425932626, 'epoch': 1.6903553299492384}
{'loss': 0.1279, 'grad_norm': 0.09248217940330505, 'learning_rate': 0.00016951924276746425, 'epoch': 2.027072758037225}
{'loss': 0.1363, 'grad_norm': 0.10165614634752274, 'learning_rate': 0.00015413923582100465, 'epoch': 2.3654822335025383}
{'loss': 0.1176, 'grad_norm': 0.09596869349479675, 'learning_rate': 0.00013667349407834833, 'epoch': 2.703891708967851}
{'loss': 0.1206, 'grad_norm': 0.09764909

In [ ]:
# Saving the model in the directory to call it for response generation

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# 1) Paths
MODEL_DIR = "./gpt_neo_fp16_lora"

In [ ]:
# 2) Load PEFT config & base model
peft_config = PeftConfig.from_pretrained(MODEL_DIR)
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.float16,
)

In [ ]:
# 3) Wrap with LoRA adapters
model = PeftModel.from_pretrained(base_model, MODEL_DIR)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoForCausalLM(
      (transformer): GPTNeoModel(
        (wte): Embedding(50257, 2048)
        (wpe): Embedding(2048, 2048)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPTNeoBlock(
            (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (attn): GPTNeoAttention(
              (attention): GPTNeoSelfAttention(
                (attn_dropout): Dropout(p=0.0, inplace=False)
                (resid_dropout): Dropout(p=0.0, inplace=False)
                (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_f

In [ ]:
# 4) Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 5) Generation function
def generate_response(query: str, max_new_tokens=100) -> str:
    prompt = f"User: {query}\nAssistant:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512,
    ).to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            no_repeat_ngram_size=2,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text.split("Assistant:")[-1].strip()


In [ ]:

# 6) Example usage
if __name__ == "__main__":
    queries = [
        "Hi! How you doing?",
    ]
    for q in queries:
        print(f"> {q}\n{generate_response(q)}\n")

    # Or enter an interactive loop:
    print("Enter 'exit' to quit.")
    while True:
        q = input("You: ").strip()
        if q.lower() in {"exit","quit"}:
            break
        print("Bot:", generate_response(q))

> Hi! How you doing?
I’m the Ecommerce and Retail Agent, here for shopping.

Enter 'exit' to quit.
You: sup
Bot: Could you please rephrase your question? I’d be happy to assist you with e-commerce platforms and online retail stores.


KeyboardInterrupt: Interrupted by user